In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt

In [0]:
'''  Upload dataset et delete labels 5,6,7 '''

def upload_data(iggy):
  x = pd.read_csv("drive/My Drive/inwr.csv")
  y = pd.read_csv("drive/My Drive/trwr.csv")

  x = x.iloc[:,1:].values
  y = y.iloc[:,[1]].values
  y = y.astype('int32')
  x = x.astype('float32')
  print(x)
  if iggy == True :
    ignore = [5,6,7]
    for i in ignore :
      fiv = np.where(y == i)
      y = np.delete(y, fiv)
      x = np.delete(x,fiv,axis=0)
  y = y.reshape((y.shape[0],1))
  return x, y

In [0]:
'''    one hot encoding for labels   '''
import keras

def target_to_cat(y):
  z = keras.utils.to_categorical(y)
  return z

In [0]:
'''    Splitting data '''

from sklearn import preprocessing
def data(x,z,sh):
  
  indices = np.arange(x.shape[0])
  X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(x, z,indices, test_size=0.33, shuffle=sh)
  
  mm_scaler = preprocessing.MinMaxScaler()
  X_train = mm_scaler.fit_transform(X_train)
  X_test = mm_scaler.fit_transform(X_test)

  
  X_train, X_val, y_train, y_val, indices_train, indices_val = train_test_split(X_train, y_train,indices_train, test_size=0.3, shuffle=sh)
  indices = np.append(indices_train, indices_test)
  return X_train, X_val, y_train, y_val, X_test, y_test

In [0]:
'''    Creation du modele :
              nplx : number of neurons per layer 
              hdx : number of hidden layers
       nplx will be multiplied by its hdx and add to the architecture
       the final architecture is the concatenation of hd layers
       Exemple : npl1 = 10 / npl2 = 32 / npl3 = 64 /
                 hd1 = 2  /  hd2 = 1  / hd3 = 2
       Architecture : 6-10-10-32-64-64-5
'''

from keras.callbacks import EarlyStopping, TerminateOnNaN

def create_model(npl1, npl2,  npl3, 
                  hd1, hd2, hd3,
                  activation,optimizer, input_shape=6, output_shape=5):
  
  
  
  model = Sequential()
  # Add an input layer
  first=True
    
  for i in range(hd1):
    if first:
      model.add(Dense(npl1, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl1, activation=activation))
      
  for i in range(hd2):
    if first:
      model.add(Dense(npl2, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl2, activation=activation))

  for i in range(hd3):
    if first:
      model.add(Dense(npl3, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl3, activation=activation))    

  model.add(Dense(5, activation=activation))
  #compile model


  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=
  ['accuracy'])
  
  return model

In [0]:
import random
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, TerminateOnNaN, ModelCheckpoint
import time

start = time.clock()
print("Tps :"+str((time.clock() - start)))

x, y = upload_data(True)

# one hot encoding
y = target_to_cat(y)
# split
X_train, X_val, y_train, y_val, X_test, y_test = data(x,y,True)

# Fonction d'activation
act = 'sigmoid'

# Fonctions d'apprentissage
optimizer = ['adam','sgd']

# nombre de couches
nl1 = [0]
nl2 = [2]
nl3 = [1]

# neuronnes par couche
nn1=[0]
nn2=[10]
nn3=[10]



bn1=0
bn2=0 
bn3=0 
bl1=0
bl2=0
bl3 = 0 
best_accuracy = 0.0
stats = []
best_val_loss = 1

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=40)
tnan = TerminateOnNaN()

for o in optimizer : 
  
  for i in range(0,1):
    l1 = random.choice(nl1)
    l2 = random.choice(nl2)
    l3 = random.choice(nl3)

    n1 = random.choice(nn1)
    n2 = random.choice(nn2)
    n3 = random.choice(nn3)
    

    print(n1,n2,n3,l1,l2,l3,act,o)

    save_clb = ModelCheckpoint(str(i)+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    model = create_model(n1,n2,n3,l1,l2,l3,act,o)

    history = model.fit(X_train, y_train , validation_data=(X_val, y_val), batch_size=128 ,epochs=50, callbacks=[es,tnan,save_clb])
    
    stats.append([str(n1),str(n2),str(n3),str(l1),str(l2),str(l3),act,o,str(history.history['val_loss'][-1]),str(history.history['acc'][-1])])
    
    
    # Courbe d'erreur
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    # Courbe de precision
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    # Evaluation de la performance du modele sur les données de test
    print(model.evaluate(X_test,y_test))
    
    # Selection du meilleur modele
    if (history.history['val_loss'][-1] < best_val_loss and history.history['acc'][-1] > best_accuracy):
          bn1, bn2, bn3, bl1, bl2, bl3 = n1,n2,n3,l1,l2,l3
          best_val_loss = history.history['val_loss'][-1]
          best_accuracy = history.history['acc'][-1]
          best_model = history
          model.save('best_model.h5')
      
    end=time.clock()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))   
    

In [0]:
from keras.models import load_model

model = load_model('0.h5')

Searching for the best architecture


Part 1 :
      With & Without one hot encoding

In [0]:
from keras.callbacks import EarlyStopping, TerminateOnNaN

def create_model(npl1, npl2,  npl3, 
                  hd1, hd2, hd3,
                  activation,optimizer, input_shape=6, output_shape=5):
  
  
  
  model = Sequential()
  # Add an input layer
  first=True
    
  for i in range(hd1):
    if first:
      model.add(Dense(npl1, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl1, activation=activation))
      
  for i in range(hd2):
    if first:
      model.add(Dense(npl2, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl2, activation=activation))

  for i in range(hd3):
    if first:
      model.add(Dense(npl3, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl3, activation=activation))    

  model.add(Dense(5, activation=activation))
  #compile model

  #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=
  ['accuracy'])
  
  return model

In [0]:
import random

x, y = upload_data(True)

X_train, X_val, y_train, y_val, X_test, y_test = data(x,y,True)
activation=['sigmoid']
optimizer = ['adam']

# numbers of layers
nl1 = [1]
nl2 = [0]
nl3 = [0]

# neurons in each layer
nn1=[5]
nn2=[5]
nn3=[5]





bn1=0
bn2=0 
bn3=0 
bl1=0
bl2=0
bl3 = 0 
best_accuracy = 0.0
best_val_loss = 1
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
tnan = TerminateOnNaN()

stats = []

for i in range(0,2):
  l1 = random.choice(nl1)
  l2 = random.choice(nl2)
  l3 = random.choice(nl3)

  n1 = random.choice(nn1)
  n2 = random.choice(nn2)
  n3 = random.choice(nn3)

  act = random.choice(activation)
  
  opt = random.choice(optimizer)
  
  print(n1,n2,n3,l1,l2,l3,act,opt)
  model = create_model(n1,n2,n3,l1,l2,l3,act,opt)
  
  history = model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=8, callbacks=[es,tnan])
  stats.append([str(n1),str(n2),str(n3),str(l1),str(l2),str(l3),act,opt,str(history.history['val_loss'][-1]),str(history.history['acc'][-1])])
  
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
    
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  
  
  if (history.history['val_loss'][-1] < best_val_loss and history.history['acc'][-1]):
        bn1, bn2, bn3, bl1, bl2, bl3 = n1,n2,n3,l1,l2,l3
        best_val_loss = history.history['val_loss'][-1]
        best_accuracy = history.history['acc'][-1]
        best_model = history
        model.save('best_model.h5')
  

In [0]:
np.savetxt("st1.csv",stats, delimiter=",", fmt='%s')

In [0]:
from google.colab import files
files.download('st1.csv') 

Part 2 :
      
      Search for the number of layers and neurons per layer
      

In [0]:
import random
from keras.callbacks import EarlyStopping, TerminateOnNaN, ModelCheckpoint

x, y = upload_data(True)
y = target_to_cat(y)

X_train, X_val, y_train, y_val, X_test, y_test = data(x,y,True)
activation=['sigmoid','softmax']
optimizer = ['adam','adagrad','adadelta','sgd']
# numbers of layers
nl1 = [1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[5,10,20,50,80,100]
nn2=[5,10,20,50,80,100]
nn3=[5,10,20,50,80,100]

stats = []


bn1=0
bn2=0 
bn3=0 
bl1=0
bl2=0
bl3 = 0 
best_accuracy = 0.0
best_val_loss = 1
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
tnan = TerminateOnNaN()

for i in range(0,10):
  l1 = random.choice(nl1)
  l2 = random.choice(nl2)
  l3 = random.choice(nl3)

  n1 = random.choice(nn1)
  n2 = random.choice(nn2)
  n3 = random.choice(nn3)
  
  
  act = random.choice(activation)
  opt = random.choice(optimizer)
  
  print(n1,n2,n3,l1,l2,l3,opt,act)
  
  save_clb = ModelCheckpoint(str(i)+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
  model = create_model(n1,n2,n3,l1,l2,l3,act,opt)
  
  history = model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=20, batch_size=512, callbacks=[es,tnan,save_clb])
  stats.append([str(n1),str(n2),str(n3),str(l1),str(l2),str(l3),act,opt,str(history.history['val_loss'][-1]),str(history.history['acc'][-1])])
  
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
    
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  
  if (history.history['val_loss'][-1] < best_val_loss and history.history['acc'][-1]):
        bn1, bn2, bn3, bl1, bl2, bl3 = n1,n2,n3,l1,l2,l3
        best_val_loss = history.history['val_loss'][-1]
        best_accuracy = history.history['acc'][-1]
        best_model = history
        model.save('best_model.h5')
  

In [0]:
np.savetxt("st2_3.csv",stats, delimiter=",", fmt='%s')
from google.colab import files
files.download('st2_3.csv') 

In [0]:
import random
from keras.callbacks import EarlyStopping, TerminateOnNaN, ModelCheckpoint

x, y = upload_data(True)
y = target_to_cat(y)
X_train, X_val, y_train, y_val, X_test, y_test = data(x,y,True)
activation=['sigmoid','softmax']
optimizer = ['adam','adagrad','adadelta']
# numbers of layers
nl1 = [1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[5,10,50,80,100]
nn2=[5,10,50,80,100]
nn3=[5,10,50,80,100]

stats = []


bn1=0
bn2=0 
bn3=0 
bl1=0
bl2=0
bl3 = 0 
best_accuracy = 0.0
best_val_loss = 1
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
tnan = TerminateOnNaN()

for i in range(0,10):
  l1 = random.choice(nl1)
  l2 = random.choice(nl2)
  l3 = random.choice(nl3)

  n1 = random.choice(nn1)
  n2 = random.choice(nn2)
  n3 = random.choice(nn3)
  
  
  act = random.choice(activation)
  opt = random.choice(optimizer)
  
  print(n1,n2,n3,l1,l2,l3,opt,act)
  
  save_clb = ModelCheckpoint(str(i)+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
  model = create_model(n1,n2,n3,l1,l2,l3,act,opt)
  
  history = model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=50, batch_size=512, callbacks=[es,tnan,save_clb])
  stats.append([str(n1),str(n2),str(n3),str(l1),str(l2),str(l3),act,opt,str(history.history['val_loss'][-1]),str(history.history['acc'][-1])])
  
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
    
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  
  if (history.history['val_loss'][-1] < best_val_loss and history.history['acc'][-1]):
        bn1, bn2, bn3, bl1, bl2, bl3 = n1,n2,n3,l1,l2,l3
        best_val_loss = history.history['val_loss'][-1]
        best_accuracy = history.history['acc'][-1]
        best_model = history
        model.save('best_model.h5')
  

In [0]:
np.savetxt("st2_3.csv",stats, delimiter=",", fmt='%s')
from google.colab import files
files.download('st2_3.csv') 

Partie 3 :
      Archi avec 100 neurones par couche (sigmoid, adam)

In [0]:
from keras.callbacks import EarlyStopping, TerminateOnNaN

def create_model(npl1, npl2,  npl3, 
                  hd1, hd2, hd3,
                  activation,lr, input_shape=6, output_shape=5):
  
  
  
  model = Sequential()
  # Add an input layer
  first=True
    
  for i in range(hd1):
    if first:
      model.add(Dense(npl1, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl1, activation=activation))
      
  for i in range(hd2):
    if first:
      model.add(Dense(npl2, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl2, activation=activation))

  for i in range(hd3):
    if first:
      model.add(Dense(npl3, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl3, activation=activation))    

  model.add(Dense(5, activation=activation))
  #compile model

  #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
  opt = keras.optimizers.Adam(lr=lr)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=
  ['accuracy'])
  
  return model

In [0]:
import random
from keras.callbacks import EarlyStopping, TerminateOnNaN, ModelCheckpoint

x, y = upload_data(True)
y = target_to_cat(y)
X_train, X_val, y_train, y_val, X_test, y_test = data(x,y,True)
act = 'sigmoid'
optimizer = ['adam']
learn=[0.1,0.01,0.001]
# numbers of layers
nl1 = [2]
nl2 = [2,3]
nl3 = [0]

# neurons in each layer
nn1=[100]
nn2=[100]
nn3=[0]

bn1=0
bn2=0 
bn3=0 
bl1=0
bl2=0
bl3 = 0 
best_accuracy = 0.0
stats = []
best_val_loss = 1
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
tnan = TerminateOnNaN()

for o in optimizer : 
  
  for i in range(0,4):
    l1 = random.choice(nl1)
    l2 = random.choice(nl2)
    l3 = random.choice(nl3)

    n1 = random.choice(nn1)
    n2 = random.choice(nn2)
    n3 = random.choice(nn3)
    lr = random.choice(learn)

    print(n1,n2,l1,l2,l3,act,lr)

    save_clb = ModelCheckpoint(str(i)+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    model = create_model(n1,n2,n3,l1,l2,l3,act,lr)

    history = model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=100, callbacks=[es,tnan,save_clb])
    stats.append([str(n1),str(n2),str(n3),str(l1),str(l2),str(l3),act,opt,str(history.history['val_loss'][-1]),str(history.history['acc'][-1])])
    import matplotlib.pyplot as plt

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    if (history.history['val_loss'][-1] < best_val_loss and history.history['acc'][-1]):
          bn1, bn2, bn3, bl1, bl2, bl3 = n1,n2,n3,l1,l2,l3
          best_val_loss = history.history['val_loss'][-1]
          best_accuracy = history.history['acc'][-1]
          best_model = history
          model.save('best_model.h5')
  

In [0]:
np.savetxt("stp3.csv",stats, delimiter=",", fmt='%s')
from google.colab import files
files.download('stp3.csv') 

In [0]:
import random
from keras.callbacks import EarlyStopping, TerminateOnNaN, ModelCheckpoint

x, y = upload_data(True)
y = target_to_cat(y)
X_train, X_val, y_train, y_val, X_test, y_test = data(x,y,True)
activation='sigmoid'
optimizer = 'adam'
# numbers of layers
nl1 = [2]
nl2 = [3]
nl3 = [0]

# neurons in each layer
nn1=[100]
nn2=[100]
nn3=[0]


bn1=0
bn2=0 
bn3=0 
bl1=0
bl2=0
bl3 = 0 
best_accuracy = 0.0
best_val_loss = 1
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
tnan = TerminateOnNaN()

for i in range(0,2):
  l1 = random.choice(nl1)
  l2 = random.choice(nl2)
  l3 = random.choice(nl3)

  n1 = random.choice(nn1)
  n2 = random.choice(nn2)
  n3 = random.choice(nn3)
  
  
  print(n1,n2,l1,l2,l3)
  
  save_clb = ModelCheckpoint(str(i)+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
  model = create_model(n1,n2,n3,l1,l2,l3,activation,optimizer)
  
  history = model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=100,batch_size=700, callbacks=[tnan,save_clb])
  
  if (history.history['val_loss'][-1] < best_val_loss and history.history['acc'][-1]):
        bn1, bn2, bn3, bl1, bl2, bl3 = n1,n2,n3,l1,l2,l3
        best_val_loss = history.history['val_loss'][-1]
        best_accuracy = history.history['acc'][-1]
        best_model = history
        model.save('best_model.h5')
  

In [0]:
from keras.models import load_model

model = load_model('best_model.h5')

In [0]:
print(model.summary())

In [0]:
model.evaluate(X_test, y_test)

In [0]:
import matplotlib.pyplot as plt

plt.plot(best_model.history['loss'])
plt.plot(best_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
from google.colab import files
files.download('best_model.h5') 


In [0]:
from keras.callbacks import EarlyStopping

def create_model(npl1=1, npl2=1,  npl3=1, 
hd1=1000, hd2=500, hd3 = 200,
activation = 'sigmoid',optimizer = 'adam', input_shape=8, output_shape=5):
  
  
  
  model = Sequential()
  # Add an input layer
  first=True
    
  for i in range(hd1):
    if first:
      model.add(Dense(npl1, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl1, activation=activation))
      
  for i in range(hd2):
    if first:
      model.add(Dense(npl2, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl2, activation=activation))

  for i in range(hd3):
    if first:
      model.add(Dense(npl3, input_dim=input_shape, activation=activation))
      first=False
    else: 
      model.add(Dense(npl3, activation=activation))    

  model.add(Dense(5, activation=activation))
  #compile model

  #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=
  ['accuracy'])
  
  return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=1000, verbose=1)

In [0]:
from sklearn.model_selection import PredefinedSplit
from keras.callbacks import EarlyStopping, TerminateOnNaN

x, y = upload_data(True)
y = target_to_cat(y)

X_train, X_val, y_train, y_val= data(x,y,True)

#cv = PredefinedSplit(indices)

# activation
activation=['sigmoid']
optimizer = ['adadelta','adam']
# numbers of layers
nl1 = [0,1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[8,16,32]
nn2=[8,16,32]
nn3=[8,16,32]

batch_size = [128,256,512,1024]

# dictionary summary
param_grid = dict(
                    npl1=nn1, npl2=nn2, npl3=nn3, hd1=nl1, hd2=nl2, hd3=nl3,
                    activation=activation,optimizer=optimizer,batch_size=batch_size,
                    input_shape=[x.shape[1]], output_shape = [y.shape[1]],
                 )

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping, TerminateOnNaN


grid = RandomizedSearchCV(estimator=model, cv=2, param_distributions=param_grid, 
                          verbose=20,  n_iter=5, n_jobs=-1)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
tnan = TerminateOnNaN()

grid_result = grid.fit(X_train, y_train,validation_data = (X_val, y_val),callbacks=[es,tnan])

In [0]:
best_parameters = grid_result.best_params_
best_accuracy = grid_result.best_score_

In [0]:
model = KerasClassifier(build_fn=create_model, verbose=1)
# activation
activation=['relu', 'sigmoid']
optimizer = ['adadelta','adam']
# numbers of layers
nl1 = [1,2,3]
nl2 = [0,1,2,3]
nl3 = [0,1,2,3]

# neurons in each layer
nn1=[10,50,80,100]
nn2=[80,100,200]
nn3=[50,20,10]

epochs = [100,500,1000]

batch_size = [200,512,1024]


param_grid = dict(
                    npl1=nl1, npl2=nl2, npl3=nl3, hd1=nn1, hd2=nn2, hd3=nn3,
                    activation=activation,optimizer=optimizer,epochs=epochs,
                    batch_size=batch_size,
                    input_shape=[x.shape[1]], output_shape = [y.shape[1]],
                 )

In [0]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=model,
                           param_grid = param_grid,
                           scoring = 'accuracy',
                           cv = 3)
grid_search = grid_search.fit(X_train, y_train, validation_data = (X_val, y_val))

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

  
x, y = upload_data(True)
y = target_to_cat(y)

X_train, X_val, y_train, y_val = data(x,y,True)
model = KerasClassifier(build_fn=create_model)

parameters = {'batch_size': [512, 20000, 120000],
              'epochs': [500, 1000, 2000],
              'optimizer': ['adam', 'rmsprop'],
              }
grid_search = GridSearchCV(estimator=model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train, validation_data = (X_val, y_val),  callbacks=[es])
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model)
optimizers = ['rmsprop', 'adam']
epochs = np.array([500, 1000, 2000])
batches = np.array([512, 20000, 120000])

param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))


In [0]:
'''   Fonction qui partitionne les données en 67 % entrainement et 33 % test   '''
def splitting(x, z):
  X_train, X_test, y_train, y_test = train_test_split(x, z, test_size=0.33, shuffle=False)
  return X_train, X_test, y_train, y_test